#패키지 및 샘플 로딩

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import statistics as ss
import warnings
warnings.filterwarnings(action='ignore')
import json
import pandas.io.json as pdjson
import ast
import gc
gc.enable()
from pandas import json_normalize

※ 훈련 세트(train)과 테스트 세트(test)를 처리하는 기본적인 방식은 같으나, 훈련세트의 파일의 용량이 3배 가량 크기 때문에 좀 더 분할 처리되도록 함수를 달리하였음 

# 훈련 세트 처리

## 함수 정의

In [ ]:
#dataframe의 json 컬럼을 파싱하고 pickle로 저장하는 함수
def parse(dataframe, usecols=None,trial=None):
    numbers = 0
    dfs = dataframe

    for df in dfs:
      ans = pd.DataFrame()
      df.reset_index(drop = True,inplace=True)
      device_list=df['device'].tolist()
      
      #deleting unwanted columns before normalizing
      for device in device_list:
          del device['browserVersion'],device['browserSize'],device['flashVersion'],device['mobileInputSelector'],device['operatingSystemVersion'],device['screenResolution'],device['screenColors']
      df['device']=pd.Series(device_list)
      
      geoNetwork_list=df['geoNetwork'].tolist()
      for network in geoNetwork_list:
          del network['latitude'],network['longitude'],network['networkLocation'],network['cityId']
      df['geoNetwork']=pd.Series(geoNetwork_list)
      
      df['hits']=df['hits'].apply(ast.literal_eval)
      df['hits']=df['hits'].str[0]
      df['hits']=df['hits'].apply(lambda x: {'index':np.NaN,'value':np.NaN} if pd.isnull(x) else x)
      
      
      df['customDimensions']=df['customDimensions'].apply(ast.literal_eval)
      df['customDimensions']=df['customDimensions'].str[0]
      df['customDimensions']=df['customDimensions'].apply(lambda x: {'index':np.NaN,'value':np.NaN} if pd.isnull(x) else x)
      
      JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource','hits','customDimensions']

      for column in JSON_COLUMNS:
          column_as_df = json_normalize(df[column])
          column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
          df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

      if usecols:
          print('--- usecols')
          usecols = set(usecols).intersection(df.columns)
          usecols = list(usecols)
          use_df = df[usecols]
          del df
          gc.collect()
          ans = pd.concat([ans,use_df],axis = 0).reset_index(drop = True)
      else:
          print('--- no usecols')
          ans = pd.concat([ans,df],axis = 0).reset_index(drop = True)
          del df
          gc.collect()

      # print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
      print('Stored shape:', ans.shape)
      numbers += 1
      print(numbers)
      ans.to_pickle(f'/content/drive/MyDrive/ds_study/ml_file/train_{trial}_{numbers}.pkl')
      del ans
      gc.collect()

In [ ]:
# 파일을 분할하여 로딩하고, 위에서 선언한 parse 함수를 통해 pickle로 저장하는 함수
def split_train(filePath, nrows=None, chunksize=None, usecols=None):
    n = 0
    count = 0
    trial = 1
    flag =True
    while flag : 
      JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
      if n == 0:
        df = pd.read_csv(csv_path,
                          skiprows=n, 
                        converters={column: json.loads for column in JSON_COLUMNS}, 
                        dtype={'fullVisitorId': 'str'}, # Important!!
                        chunksize=chunksize,
                        nrows=nrows
                        )
      else:
        df = pd.read_csv(csv_path,
                  skiprows=[ i for i in range(1,n)], 
                converters={column: json.loads for column in JSON_COLUMNS}, 
                dtype={'fullVisitorId': 'str'}, # Important!!
                chunksize=chunksize,
                nrows=nrows
                )

      print('================================File load complete')

      #위에서 선언한 parse 함수 호출
      sample = parse(df, trial = trial)

      print('================================File save complete')

      count +=1
      n +=nrows
      trial +=1
      del (df)
      if n > 1708337:
        flag=False     


In [ ]:
# 여러개로 쪼개서 저장된 pickle 파일을 로드하고, 합쳐서 dataframe으로 반환하는 함수
def concat_pickle(path, start = None, end = None): #srart : 몇 번 파일 부터 불러올지 , end : 몇 번 파일까지 불러올지
    
    start = start
    end = end

    for t in range(start, end+1):
      if t==171:
        for n in range(1,10):
          tmp = pd.read_pickle(f'{path}/train{t}_{n}.pkl')
          dfs = pd.concat([dfs,tmp],axis=0)
          del tmp
          print(f'========train{t}_{n} to df')
          print('========fin')
      else:
        for n in range(1,11):
            if t == start and n== 1:
                dfs = pd.read_pickle(f'{path}/train{t}_{n}.pkl')
                print(f'========train{t}_{n} to df')
            else : 
                tmp = pd.read_pickle(f'{path}/train{t}_{n}.pkl')
                dfs = pd.concat([dfs,tmp],axis=0)
                del tmp
                print(f'========train{t}_{n} to df')
    
    return dfs

## 변환 및 저장

In [ ]:
# train 데이터 로드, json 컬럼 파싱, 피클 저장
csv_path='/content/drive/MyDrive/data/train_v2.csv'

split_train(csv_path,nrows=10000,chunksize=1000)

In [ ]:
# 커널이 끊길 수 있어, 피클로 저장된 총 171개로 저장된 파일을 4번에 나눠서 병합

# 1) 1~50번 째 파일까지 로드해서 병합 후 피클로 저장

path= '/content/drive/MyDrive/ds_study/ml_file'

df = concat_pickle(path, start=1, end=50)

df.to_pickle(f'/content/drive/MyDrive/ds_study/ml_file/train_fin_01.pkl')

del df

In [ ]:
# 2) 51 ~ 200번 째 파일까지 로드해서 병합 후 피클로 저장
df2 = concat_pickle(path, start=51, end=100)

df2.to_pickle(f'/content/drive/MyDrive/ds_study/ml_file/train_fin_02.pkl')

del df2

In [ ]:
# 3) 101 ~ 150번 째 파일까지 로드해서 병합 후 피클로 저장

df3 = concat_pickle(path, start=101, end=150)

df3.to_pickle(f'/content/drive/MyDrive/ds_study/ml_file/train_fin_03.pkl')

del df3

In [ ]:
# 3) 151 ~ 171번 째 파일까지 로드해서 병합 후 피클로 저장
df4 = concat_pickle(path, start=151, end=171)

df4.to_pickle(f'/content/drive/MyDrive/ds_study/ml_file/train_fin_04.pkl')

del df4

In [ ]:
# 1) 2)번 파일 병합 후 저장
df1 = pd.read_pickle('/content/drive/MyDrive/ds_study/ml_file/train_fin_01.pkl')
print(f'===train_fin_1 load complete')

df2 = pd.read_pickle('/content/drive/MyDrive/ds_study/ml_file/train_fin_02.pkl')
print(f'===train_fin_2 load complete')

df1 = pd.concat([df1,df2],axis=0)

del df2

df1.to_pickle(f'/content/drive/MyDrive/ds_study/ml_file/train_fin_fin_01.pkl')

del df1


In [ ]:
# 3) 4)번 파일 병합 후 저장
df3 = pd.read_pickle('/content/drive/MyDrive/ds_study/ml_file/train_fin_03.pkl')
print(f'===train_fin_3 load complete')

df4 = pd.read_pickle('/content/drive/MyDrive/ds_study/ml_file/train_fin_04.pkl')
print(f'===train_fin_4 load complete')

df3 = pd.concat([df3,df4],axis=0)

del df4

df3.to_pickle(f'/content/drive/MyDrive/ds_study/ml_file/train_fin_fin_02.pkl')

del df3


In [ ]:
# 최종 파일 병합 후 저장
df1 = pd.read_pickle('/content/drive/MyDrive/ds_study/ml_file/train_fin_fin_01.pkl')

df2 = pd.read_pickle('/content/drive/MyDrive/ds_study/ml_file/train_fin_fin_02.pkl')

df1 = pd.concat([df1,df2],axis=0)

del df2

df1

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device_browser,device_operatingSystem,device_isMobile,...,trafficSource_campaignCode,hits_transaction.transactionId,hits_transaction.transactionRevenue,hits_transaction.transactionTax,hits_transaction.transactionShipping,hits_transaction.affiliation,hits_transaction.localTransactionRevenue,hits_transaction.localTransactionTax,hits_transaction.localTransactionShipping,hits_item.transactionId
0,Organic Search,20171016,3162355547410993243,Not Socially Engaged,1508198450,1,1508198450,Firefox,Windows,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Referral,20171016,8934116514970143966,Not Socially Engaged,1508176307,6,1508176307,Chrome,Chrome OS,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Direct,20171016,7992466427990357681,Not Socially Engaged,1508201613,1,1508201613,Chrome,Android,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Organic Search,20171016,9075655783635761930,Not Socially Engaged,1508169851,1,1508169851,Chrome,Windows,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Organic Search,20171016,6960673291025684308,Not Socially Engaged,1508190552,1,1508190552,Chrome,Windows,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,Social,20170104,5123779100307500332,Not Socially Engaged,1483554750,1,1483554750,Chrome,Windows,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
334,Social,20170104,7231728964973959842,Not Socially Engaged,1483543798,1,1483543798,Chrome,Android,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
335,Social,20170104,5744576632396406899,Not Socially Engaged,1483526434,1,1483526434,Android Webview,Android,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
336,Social,20170104,2709355455991750775,Not Socially Engaged,1483592857,1,1483592864,Chrome,Windows,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df1.reset_index(drop=True, inplace=True)

In [ ]:
df1.drop(9999,axis=0,inplace=True) #9999번과 10000번의 row가 중복되어, 9999번 row를 삭제

In [ ]:
df1.shape

(1708337, 128)

In [ ]:
#최종 저장
df1.to_pickle('/content/drive/MyDrive/data/train_02_fin.pkl')

# 테스트 세트 처리

## 함수 정의

In [ ]:
def parse(data_Frame, nrows=None, chunksize=None, usecols=None):

    # JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    ans = pd.DataFrame()

    # dfs = pd.read_csv(csv_path, 
    #                  converters={column: json.loads for column in JSON_COLUMNS}, 
    #                  dtype={'fullVisitorId': 'str'}, # Important!!
    #                  chunksize=chunksize,
    #                  nrows=nrows)
    for df in data_Frame:
      df.reset_index(drop = True,inplace=True)

      device_list=df['device'].tolist()
      
      #deleting unwanted columns before normalizing
      for device in device_list:
          del device['browserVersion'],device['browserSize'],device['flashVersion'],device['mobileInputSelector'],device['operatingSystemVersion'],device['screenResolution'],device['screenColors']
      df['device']=pd.Series(device_list)
      
      geoNetwork_list=df['geoNetwork'].tolist()
      for network in geoNetwork_list:
          del network['latitude'],network['longitude'],network['networkLocation'],network['cityId']
      df['geoNetwork']=pd.Series(geoNetwork_list)
      
      df['hits']=df['hits'].apply(ast.literal_eval)
      df['hits']=df['hits'].str[0]
      df['hits']=df['hits'].apply(lambda x: {'index':np.NaN,'value':np.NaN} if pd.isnull(x) else x)
      
      
      df['customDimensions']=df['customDimensions'].apply(ast.literal_eval)
      df['customDimensions']=df['customDimensions'].str[0]
      df['customDimensions']=df['customDimensions'].apply(lambda x: {'index':np.NaN,'value':np.NaN} if pd.isnull(x) else x)
      
      JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource','hits','customDimensions']

      for column in JSON_COLUMNS:
          column_as_df = json_normalize(df[column])
          column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
          df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

      if usecols:
          print('--- usecols')
          usecols = set(usecols).intersection(df.columns)
          usecols = list(usecols)
          use_df = df[usecols]
          del df
          gc.collect()
          ans = pd.concat([ans,use_df],axis = 0).reset_index(drop = True)
      else:
          print('--- no usecols')
          ans = pd.concat([ans,df],axis = 0).reset_index(drop = True)
          del df
          gc.collect()

      # print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
      print('Stored shape:', ans.shape)
    return ans

print("The 'parse' function to flatten JSON columns have been created")

In [ ]:
def split_train(filePath, fileName, nrows=None, chunksize=1000, limit=1708337):
    
    n = 0
    count = 0
    flag =True
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    while flag:
        df = pd.read_csv(filePath + fileName, 
                         converters={column: json.loads for column in JSON_COLUMNS}, 
                         dtype={'fullVisitorId': 'str'}, # Important!!
                         chunksize=chunksize,
                         skiprows=n, nrows=nrows)
##############
        sample = parse(df, nrows=nrows, chunksize=None, usecols=None)

        # def parse(csv_path, nrows=None, chunksize=None,usecols=None):
##############

        sample.to_pickle(f'{fileName[:-4]}_{count}.pkl')
        print(f'saving : {fileName[:-4]}_{count}.pkl')

        count += 1
        n += nrows
        if n > limit:
            flag=False

        del (df)

## 변환 및 저장

In [ ]:
split_train('D:\ml_data\Google_Analytics_Customer_Revenue_Prediction/', 'test_v2.csv', nrows=401589, limit=401589, chunksize=1000)

# 샘플링

In [ ]:
#처리를 통해 피클화한 train과 test데이터 로드
data_tr = pd.read_pickle('/content/drive/MyDrive/data/train_02_fin.pkl')
data_ts = pd.read_pickle('/content/drive/MyDrive/data/test_v2_0.pkl')

In [ ]:
# y값 결측치 처리, 타입 변환
data_tr['totals_transactionRevenue'].fillna(0, inplace=True)
data_ts['totals_transactionRevenue'].fillna(0, inplace=True)
data_tr.totals_transactionRevenue = data_tr.totals_transactionRevenue.astype('float')
data_ts.totals_transactionRevenue = data_ts.totals_transactionRevenue.astype('float')

In [ ]:
# 랜덤한 수치형 y 열 생성
np.random.seed(42)

# y 열을 10개의 bin으로 분할하여 label을 할당
data_tr['label'] = pd.cut(data_tr['totals_transactionRevenue'], bins=10, labels=False)

train_sample = data_tr.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=0.4))

train_sample.drop('label',axis=1,inplace=True)

train_sample.drop('trafficSource_campaignCode',axis=1,inplace=True)

In [ ]:
# train과 test 샘플 위아래로 결합
data = train_sample.append(data_ts)
data.reset_index(drop=True,inplace=True)

In [ ]:
#피클로 저장
data.to_pickle('/content/drive/MyDrive/data/co_data/sample_01.pkl')